<a href="https://colab.research.google.com/github/ducanha2k46/Exam/blob/main/Nho%CC%81m_5_Association_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Xử lý dữ liệu

## Khai báo thư viện

In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

## Đọc dữ liệu

In [2]:
df = pd.read_excel('/content/Online Retail.xlsx')

print("\nDữ liệu:")
df.head(10)


Dữ liệu:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


Danh sách giao dịch:
- Đã được chuyển đổi theo dạng InvoiceNo và các Items giao dịch tương ứng.


In [3]:
transactions = df.groupby('InvoiceNo')['StockCode'].apply(list).reset_index(name='items')


print("\nSố lượng giao dịch:")
print(len(transactions))

print("\nDanh sách giao dịch:")
print(transactions.head())


Số lượng giao dịch:
25900

Danh sách giao dịch:
  InvoiceNo                                              items
0    536365  [85123A, 71053, 84406B, 84029G, 84029E, 22752,...
1    536366                                     [22633, 22632]
2    536367  [84879, 22745, 22748, 22749, 22310, 84969, 226...
3    536368                       [22960, 22913, 22912, 22914]
4    536369                                            [21756]


Chuyển danh sách giao dịch về ma trận nhị phân.
Mỗi cột là mã sản phẩm:
- True: Có mua
- False: Không mua

In [4]:
transactions['items'] = transactions['items'].apply(lambda x: [str(item) for item in x])

te = TransactionEncoder()
te_ary = te.fit(transactions['items']).transform(transactions['items'])
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)
print("\nMa trận nhị phân:")
print(df_encoded.head())


Ma trận nhị phân:
   10002  10080  10120  10123C  10123G  10124A  10124G  10125  10133  10134  \
0  False  False  False   False   False   False   False  False  False  False   
1  False  False  False   False   False   False   False  False  False  False   
2  False  False  False   False   False   False   False  False  False  False   
3  False  False  False   False   False   False   False  False  False  False   
4  False  False  False   False   False   False   False  False  False  False   

   ...      M   PADS   POST      S  gift_0001_10  gift_0001_20  gift_0001_30  \
0  ...  False  False  False  False         False         False         False   
1  ...  False  False  False  False         False         False         False   
2  ...  False  False  False  False         False         False         False   
3  ...  False  False  False  False         False         False         False   
4  ...  False  False  False  False         False         False         False   

   gift_0001_40  gift_000


# 🧠 Giải thích các chỉ số trong Association Rule Mining

## 1. Antecedents và Consequents
- **Antecedents** (*tiền đề*): Là tập mục (itemset) xuất hiện trước trong một luật kết hợp.
- **Consequents** (*hệ quả*): Là tập mục xuất hiện sau, khi điều kiện của antecedents được thỏa mãn.

**Ví dụ:**  
Nếu luật là `{sữa} => {bánh mì}`, thì:
- Antecedent: `{sữa}`
- Consequent: `{bánh mì}`

---

## 2. Support (Độ hỗ trợ)
- Là **tần suất** xuất hiện của cả antecedent và consequent trong toàn bộ tập dữ liệu.

**Công thức:**
```
Support(A ⇒ B) = Số giao dịch chứa cả A và B / Tổng số giao dịch
```

**Ý nghĩa:**  
Luật này có phổ biến trong dữ liệu hay không.

---

## 3. Confidence (Độ tin cậy)
- Là xác suất mà Consequent xảy ra khi Antecedent đã xảy ra.

**Công thức:**
```
Confidence(A ⇒ B) = Support(A ∪ B) / Support(A)
```

**Ý nghĩa:**  
Khi A xảy ra, khả năng B xảy ra là bao nhiêu phần trăm.

---

## 4. Lift (Độ nâng)
- Đo lường mức độ **liên kết thực sự** giữa A và B so với khi chúng độc lập.

**Công thức:**
```
Lift(A ⇒ B) = Confidence(A ⇒ B) / Support(B)
```

**Ý nghĩa:**
- **Lift = 1**: A và B **độc lập**, không ảnh hưởng đến nhau.
- **Lift > 1**: A và B **có mối quan hệ tích cực** (đồng xuất hiện nhiều hơn mong đợi).
- **Lift < 1**: A và B **có mối quan hệ tiêu cực** (ít xảy ra cùng nhau hơn mong đợi).




# APRIORI ALGORITHM

Các tập hợp mặt hàng thường xuyên \
Áp dụng thuật toán Aprioti để tìm tập hợp mặt hàng thường xuyên \
min_support=0.015 (xuất hiện trong ít nhất 1.5% giao dịch)

In [ ]:
frequent_itemsets = apriori(df_encoded, min_support=0.01, use_colnames=True)

print("\nCác tập hợp mặt hàng thường xuyên:")
print(frequent_itemsets)

Tạo luật kết hợp từ tập hợp thường xuyên \\
min_threshold=0.3 (confidence tối thiểu 30%)

In [ ]:
if frequent_itemsets.empty:
    print("Không tìm thấy tập hợp mặt hàng thường xuyên nào với min_support=0.015.")
else:
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)
    if rules.empty:
        print("\nKhông tìm thấy luật kết hợp nào với min_threshold=0.3.")
    else:
        print("\nCác luật kết hợp (Apriori):")
        print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])
        rules_sorted = rules.sort_values(['confidence', 'lift'], ascending=[False, False])
        print("\nCác luật kết hợp được sắp xếp:")
        print(rules_sorted[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

# FP-Growth

Các tập hợp mặt hàng thường xuyên \\
Áp dụng thuật toán FP-Growth để tìm tập hợp mặt hàng thường xuyên \\
min_support=0.015 (xuất hiện trong ít nhất 1% giao dịch)

In [ ]:
frequent_itemsets = fpgrowth(df_encoded, min_support=0.01, use_colnames=True)

print("\nCác tập hợp mặt hàng thường xuyên (FP-Growth):")
print(frequent_itemsets)

Tạo luật kết hợp từ tập hợp thường xuyên \\
min_threshold=0.3 (confidence tối thiểu 30%)

In [ ]:
if frequent_itemsets.empty:
    print("Không tìm thấy tập hợp mặt hàng thường xuyên nào với min_support=0.015")
else:

    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.01)

    if rules.empty:
        print("\nKhông tìm thấy luật kết hợp nào với min_threshold=0.3.")
    else:
        print("\nCác luật kết hợp (FP-Growth):")
        print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

        rules_sorted = rules.sort_values(['confidence', 'lift'], ascending=[False, False])
        print("\nCác luật kết hợp được sắp xếp:")
        print(rules_sorted[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


# 🔍 So sánh kết quả của Apriori và FP-Growth

## a. Số lượng luật
- **Apriori**: 192 luật.  
- **FP-Growth**: 429 luật.  

**Nhận xét**: FP-Growth tạo ra nhiều luật hơn đáng kể. Điều này có thể do FP-Growth hiệu quả hơn trong việc xử lý dữ liệu và tìm kiếm các tập hợp mục thường xuyên (frequent itemsets), đặc biệt với các tập dữ liệu lớn hoặc phức tạp.

---

## b. Support
- **Apriori**: Support dao động từ `0.015058` đến `0.021197`.  
- **FP-Growth**: Support dao động từ `0.018919` đến `0.021197`.  

**Nhận xét**: FP-Growth tìm được các luật với support cao hơn (tối thiểu `0.018919` so với `0.015058` của Apriori).

---

## c. Confidence
- **Apriori**: Confidence từ `0.300122` đến `0.894137`.  
- **FP-Growth**: Confidence từ `0.304726` đến `0.894137`.  

**Nhận xét**: Cả hai thuật toán cho ra các luật với confidence tương đương

---

## d. Lift

**Nhận xét**: FP-Growth tìm được các luật với lift cao hơn nhiều (tối đa `60.260387` so với `22.614223` của Apriori). Điều này cho thấy FP-Growth có khả năng phát hiện các mối quan hệ mạnh hơn giữa các sản phẩm.

---

## e. Các luật cụ thể
Một số luật xuất hiện ở cả hai kết quả, ví dụ:  
- `(22699, 22698) → (22697)` có ở cả Apriori (luật 187) và FP-Growth (luật 700), với các giá trị support, confidence, và lift gần giống nhau.

Tuy nhiên, FP-Growth tìm được các luật phức tạp hơn với nhiều mục hơn ở vế trái, ví dụ:  
- `(22699, 22423, 22698) → (22697)`, điều mà Apriori không tìm thấy trong 192 luật của nó.
